# Diplodatos 2025
## M18: Lectura Distante de Canciones Argentinas
### El proyecto

Este proyecto busca ofrecer un primer acercamiento al procesamiento del lenguaje natural (NLP), el área del machine learning que se encarga del lenguaje. Para eso, vamos a usar como "excusa" un dataset con alrededor de 30.000 letras de canciones argentinas. También se puede pensar al revés: el objetivo principal es hacer un análisis cultural con datos y la herramienta indicada para eso es el NLP.

En la práctica, el énfasis depende de ustedes: pueden tratar de entrenar un modelo muy efectivo, usando las técnicas más sofisticadas que veamos; o pueden entrenar modelos sencillos pero más interpretables, que permitan "generar conocimiento".

### "Lectura distante"
Este es un término acuñado por Franco Moretti en un ensayo del 2000, titulado ["Conjectures on World Literature"](https://warwick.ac.uk/fac/arts/english/currentstudents/undergraduates/modules/fulllist/special/globalnovel/moretti-conjectures-nlr_1.pdf). Moretti es un especialista en literaturas comparadas y pionero de las "digital humanities" que a la vuelta del siglo pasado entendió que para estudiar realmente el sistema literario mundial sin restringirse a un puñado de "clásicos" no alcanza con "leer más", sino que es necesario desarrollar nuevos métodos. Lo que él propone es un tipo de lectura (o de no-lectura) llamada "distant reading", que consiste en analizar conjuntos masivos de textos usando las herramientas de la estadística. O sea, ya no leemos detalladamente una obra en particular tratando de interpretar su significado -como en la tradición hermenéutica- sino que tomamos como objeto de estudio un corpus de textos del cual extraemos métricas de distinto tipo.

### Los trabajos
- Para el TP 1, vamos a hacer un análisis exploratorio de los datos más bien clásico. También vamos a elegir la variable "target" que va a guiar el trabajo de los demás TPs.
- Para el TP 2, vamos a llevar el análisis un poco más allá, poniendo en práctica conceptos como "bolsa de palabras", "tokenización" y "TF-IDF". Vamos a convertir el texto en vectores y estos vectores nos van a servir para hacer un modelado de tópicos (LDA), y también para entrenar los modelos del TP 3.
- Este TP es más técnico: acá vamos a entrenar modelos con distintos parámetros y distintos tipos de vectorización, y vamos a comparar los resultados en términos de performance e interpretabilidad. También se va a poder optar por un clustering.

### Introducción al dataset

El dataset contiene letras de canciones extraídas del sitio web La Cuerda (https://lacuerda.net/), junto con información proveniente de la api de Spotify: https://api.spotify.com.

Las variables son:

- **song_name**: El nombre del tema.
- **artist_name**: El nombre del artista.
- **lyrics**: La letra del tema tal como fue subida a La Cuerda.
- **estilo**: El estilo o "género" del tema. Puede ser "bal" (balada), "can" (cantautor), "gru" (grupero), "pop", "rel" (religioso), "rok" (rock), "rom" (romántico), "tra" (tradicional), "tro" (tropical).
- **album**: El nombre del álbum (Spotify).
- **release_date**: Fecha de lanzamiento del álbum (Spotify).
- **track_name**: El nombre del tema según la api de Spotify (puede no coincidir).
- **duration_ms**: La duración del tema en milisegundos (Spotify).
- **popularity**: La popularidad del tema según el algoritmo de Spotify (https://developer.spotify.com/documentation/web-api/reference/get-an-artists-top-tracks).

In [60]:
import pandas as pd

df = pd.read_csv("dataset.csv", index_col = 0)
df.head(3)

,song_name,artist_name,lyrics,estilo,song_url,album,release_date,track_name,duration_ms,popularity
0,A Mis Amigos,Alberto Cortez,A MIS AMIGOS LES ADEUDO LA TERNURA<br>Y LAS PA...,bal,https://acordes.lacuerda.net/alberto_cortez/a_...,Lo Cortez No Quita Lo Cabral (En Vivo),1994-06-09,A Mis Amigos - En Vivo,311600.0,37.0
1,A Partir De Maniana,Alberto Cortez,A partir de mañana empezaré a vivir la mitad d...,bal,https://acordes.lacuerda.net/alberto_cortez/a_...,NaN,NaN,NaN,NaN,NaN
2,Amor Desolado,Alberto Cortez,Yo puse el esfuerzo y ella la desgana<br>yo el...,bal,https://acordes.lacuerda.net/alberto_cortez/am...,Grandes Éxitos: Alberto Cortez,2005-03-21,El amor desolado,242066.0,32.0


### Consignas TP 1

1) Descripción del dataset.
    - Tamaño.
    - Registros duplicados (duplicados reales, falsos duplicados).
    - Tipos de variables.

2) Descripción de las variables (exceptuando `song_url`).
    - Medidas descriptivas.
    - Distribución (graficar).
    - ¿Hay valores nulos? ¿Qué porcentaje representan?
    - ¿Hay outliers? ¿Cuáles son? ¿Se trata de errores, o a qué pueden deberse los valores extremos?

3) Elegir una variable target, que puede ser: `estilo`, `release_date` y `popularity`. En función de esta variable se va a hacer el análisis bivariado, y se va a ajustar el modelo de clasificación del TP 3. Luego:
    - Justificar la elección de dicha variable. ¿Qué interés le ven?
    - Hacer un análisis bivariado para la variable elegida junto con una numérica y una categórica. Para numérica vs. numérica calcular correlación de Pearson y hacer un scatter plot. Para categórica vs. numérica, reportar en una tabla media, mediana y desvío estándar para cada categoría, y hacer un box-plot. Para categórica vs. categórica, hacer una tabla de contingencia y un gráfico de barras apiladas. Interpretar sucintamente los resultados.
    - Calcular el [type token ratio](https://www.reuneker.nl/files/ld/#) promedio usando una ventana deslizante de tamaño 30 y analizar la relación entre esta nueva variable y la variable target. ¡Ojo! Asegurarse de que la variable `lyrics` esté razonablemente limpia antes de hacer el análisis (esto supone eliminar registros con letras nulas, borrar términos o versos enteros que no formen parte de la letra, etc.). El criterio de limpieza va a ser fundamental para los próximos TPs. Pueden crear variables como la longitud de la letra para detectar errores en el dataset. ¿Sirve el TTR como método de detección de outliers? 
    - (Opcional) Hacer lo mismo calculando otras métricas a partir de las canciones: longitud promedio del verso, longitud del título, longitud promedio de cada palabra.

4) Elegir el resultado más significativo que surja del análisis y buscar la manera más clara de presentarlo (gráfico + descripción). Indicar si los resultados eran o no previsibles.

### Fechas importantes
- 30/06: TP 1
- 25/07: TP 2
- 11/08: Video intermedio
- 15/09: TP 3
- 29/09: Video final
- 05/12 y 06/12: Jornada de presentación de videos finales

**Las entregas se realizan vía campus.**

### Papers para chusmear

- ["Rhyme and Style Features for Musical Genre Classification by Song Lyrics" (2008)](https://www.researchgate.net/profile/Rudolf-Mayer/publication/220723682_Rhyme_and_Style_Features_for_Musical_Genre_Classification_by_Song_Lyrics/links/54b7c18d0cf2e68eb2804a33/Rhyme-and-Style-Features-for-Musical-Genre-Classification-by-Song-Lyrics.pdf). Paper que prueba la efectividad de diferentes features (temáticas, estilométricas, sintácticas, rítmicas) para predecir el género de una canción. Entre ellas usan una variante del TTR. De acá se pueden sacar ideas para el TP 2.
- ["The Language of Lyrics: An Analysis of Popular _Billboard_ Songs Across Conditions of Social and Economic Threat" (2009)](https://sci-hub.se/https://journals.sagepub.com/doi/10.1177/0261927x09335259). Paper que analiza las letras de los temas nº1 del ranking de la revista _Billboard_ desde 1955 a 2003. La hipótesis que guía este trabajo es que hay una correlación entre la situación social y las temáticas de las canciones. Ellos detectan, por ejemplo, que un aumento en el índice GHTM (General Hard Times Measure) correlaciona positivamente con la cantidad de pronombres en segunda persona y la longitud del verso.
- _Falso movimiento. El giro cuantitativo en el estudio de la literatura_, de Franco Moretti (2023, EDUNTREF). Una compilación de ensayos de Moretti traducidos al español.